# Amigos por idade

Contar a quantidade média de amigos por idade.


<img src="https://insight.ieeeusa.org/wp-content/uploads/sites/3/2018/08/network-1200-1200x700.jpg" class="bg-primary mb-1" width="300px">

## Conectando o Drive ao Colab

A primeira coisa que você quer fazer quando está trabalhando no Colab é montar seu Google Drive. Isso permitirá que você acesse qualquer diretório em seu Drive dentro do notebook Colab.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Instalando o PySpark no Google Colab

Instalar o PySpark não é um processo direto como de praxe em Python. Não basta usar um pip install apenas. Na verdade, antes de tudo é necessário instalar dependências como o Java 8, Apache Spark 2.3.2 junto com o Hadoop 2.7.

In [2]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

A próxima etapa é configurar as variáveis de ambiente, pois isso habilita o ambiente do Colab a identificar corretamente onde as dependências estão rodando.

Para conseguir “manipular” o terminal e interagir como ele, você pode usar a biblioteca os.

In [3]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

## Implementando a aplicação

A primeira coisa a fazer é criar a aplicação e a sessão do Spark.

In [6]:
from pyspark.sql import SparkSession

# A function that splits a line of input into (age, numFriends) tuples.
def parse_line(line):
    # Split by commas
    fields = line.split(",")

    # Extract the age and numFriends fields, and convert to integers
    age = int(fields[2])
    num_friends = int(fields[3])

    # Create a tuple that is our result.
    return age, num_friends

spark = (SparkSession
         .builder
         .appName("FriendsByAge")
         .getOrCreate())

Carregar cada linha dos dados de origem em um RDD

In [10]:
lines = spark.sparkContext.textFile("/content/drive/MyDrive/Ensino/IFG/2022/01/Big_Data/pos-ia-bd/04-spark/data/fakefriends.csv")

Use nossa função `parseLines` para converter em tuplas (age, numFriends)

In [11]:
rdd = lines.map(parse_line)

Muita coisa acontecendo aqui...

Estamos começando com um RDD de forma (idade, numAmigos) onde idade é a CHAVE e numAmigos é o VALOR

Usamos `mapValues` para converter cada valor `numFriends` em uma tupla de (numFriends, 1)

Em seguida, usamos `reduceByKey` para somar o total de `numFriends` e o total de instâncias para cada idade, somando todos os valores numFriends e 1s respectivamente.

In [12]:
totals_by_age = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

Então agora temos tuplas de (idade, (totalFriends, totalInstances))

Para calcular a média dividimos totalFriends/totalInstances para cada idade.

In [13]:
average_by_age = totals_by_age.mapValues(lambda x: x[0] / x[1]).sortByKey()

Colete os resultados do RDD

Isso inicia a computação do DAG e realmente executa o trabalho

In [ ]:
results = average_by_age.collect()
for r in results:
  print(r)